In [1]:
import pandas as pd
from utils import *

In [2]:
PATH_DATA = './../../data/'
PATH_PQRQUET= 'parquet/'
FILE_20 = 'quotes-2020.json.bz2'
FILE_PARQUET = 'speaker_attributes.parquet'
FILE_QID_QB = 'wikidata_labels_descriptions_quotebank.csv.bz2'
CHUNK_SIZE = 1e5

In [3]:
# Quotes for 2020
df_20 = pd.read_json(PATH_DATA+FILE_20, lines=True, compression='bz2', chunksize=CHUNK_SIZE)
# | quoteID | qotation | speaker | qids | date | numOccurrences | probas | urls | phase |

In [4]:
# Writers' Data
df_parquet = pd.read_parquet(PATH_DATA+PATH_PQRQUET+FILE_PARQUET)
# | aliases | date_of_bitrh | nationality | gender | lastrevid | ethnic_group | US_congress_bio_ID | occupation | party | academic_degree | id | label | candidacy | type | religion |

In [5]:
# Meaning of QID
df_qid = pd.read_csv(PATH_DATA+PATH_PQRQUET+FILE_QID_QB, compression='bz2')
# | QID | Label | Description |

In [133]:
# drop writers with no gender:
df_parquet_noNa_genrer = df_parquet.dropna(subset=['gender'])

# list of unique gender's qid in writers data:
writers_gender_qid = get_unique_list(df_parquet_noNa_genrer.gender)

In [135]:
# list of gender in Parquet, and its description:
writer_gender = []
writer_gender_desc = []
# QID in data but not in parquet :
bad_gender_id = []
# ex:
# Q15145782='transgender female'
# Q15145783='transgender male' !! Q2449503 is in list and also refers 'transgender male': index 5
for gender in writers_gender_qid:
    values = df_qid[df_qid.QID == gender].values
    if values.shape[0]>0:
        writer_gender.append(values[0][1])
        writer_gender_desc.append(values[0][2])
    else:
        bad_gender_id.append(gender)

dic_gender = {'gender':writer_gender, 'description':writer_gender_desc}
df_gender = pd.DataFrame(dic_gender)

print(f'gender ID in data but not in Parquet: \n{bad_gender_id}')
df_gender

## !!!
## Take care of rows: | 17 | 25 | 26 | 36 | in df_gender

gender ID in data but not in Parquet: 
['Q15145782', 'Q15145783']


,gender,description
0,male,"to be used in ""sex or gender"" (P21) to indicat..."
1,female,"to be used in ""sex or gender"" (P21) to indicat..."
2,transgender female,female person who was assigned a different gen...
3,intersex,innate variations in sex characteristics such ...
4,eunuch,castrated male human
5,transgender male,person assigned to the female sex at birth who...
6,non-binary,range of gender identities that are not exclus...
7,cisgender female,female person who was assigned female at birth
8,genderqueer,range of gender identities that are not exclus...
9,transgender person,person whose gender identity is different from...


In [162]:
# Are all writers in parquet:
df_20 = pd.read_json(PATH_DATA+FILE_20, lines=True, compression='bz2', chunksize=CHUNK_SIZE)

count_men = 0
count_women = 0
count_orhers = 0
count_unknown = 0
count_none = 0
count_all = 0

qid_men = writers_gender_qid[0]
qid_women = writers_gender_qid[1]

'''
for chunk in df_20:
    # print(df_parquet[df_parquet.id == chunk.qids])
    chunk_qid = chunk[chunk.qids.apply(lambda x: bool(x))]
    for qid in chunk_qid.qids.values:
        parquet = df_parquet[df_parquet.id == qid[0]]  ## index 340 is neither none or full -> empty list: [] ##
        print(i, parquet.gender.values)
        gender = parquet.gender.values[0]
        if gender:
            if gender[0] == qid_men:
                count_men += 1
            elif gender[0] == qid_women:
                count_women += 1
            elif gender[0] in writers_gender_qid:
                count_orhers += 1
            else:
                count_unknown += 1
        else:
            count_none += 1
            
        count_all += 1
    # print(chunk.qids.isin(df_parquet.id))
    # print(chunk.qids)
    # not_in_parquet = get_unique_list(chunk_nona.qids)
    break
'''

for chunk in df_20:
    chunk_qid = chunk[chunk.qids.apply(lambda x: bool(x))]
    print((chunk_qid.qids.values == []).sum())
    break

<ipython-input-162-bf8ced3bfe35>:43: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print((chunk_qid.qids.values == []).sum())


AttributeError: 'bool' object has no attribute 'sum'

In [ ]:
print(f'men: {count_men}\n \
women: {count_women}\n \
others: {count_others}\n \
unknown: {count_unknown}\n \
none: {count_none}\n \
all: {count_all}\n')